In [1]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Specify the model name and revision
model_name = "google/flan-t5-large"
model_revision = "main"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, revision=model_revision)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, revision=model_revision)

# Load the summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Use a library like requests to fetch the webpage content
import requests
from bs4 import BeautifulSoup

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
def info(id):
    url = f"https://parts.igem.org/Part:{id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    article_text = soup.find("div", id="mw-content-text").get_text()

    # Set the maximum length of the summary
    max_length = 25  # Adjust this value as needed

    # Generate the summary
    summary = summarizer(article_text, max_length=max_length, min_length=15, do_sample=False)

    return summary[0]["summary_text"]

In [3]:
# Example
info("BBa_I723020")

'PU promoter is a member of the XylR promoter family.'

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [5]:
with open("/kaggle/input/biobricks/parts.txt") as f:
    text = f.read()

parts_raw = [x.strip() for x in text.split(">")][1:]
parts = []
desc_lens = {}

for p in parts_raw:
    id = p.split(maxsplit=1)[0]
    part_type = p.split("\n", maxsplit=1)[0].split()[3]
    description = p.split('"', 1)[1].split('"', 1)[0]

    if len(description) not in desc_lens:
        desc_lens[len(description)] = 0
    desc_lens[len(description)] += 1

    parts.append({
        "id": id,
        "type": part_type,
        "description": description
    })


# Create a plot on the distribution of description lengths

# import matplotlib.pyplot as plt

# plt.bar(desc_lens.keys(), desc_lens.values())
# plt.xlabel("Description length")
# plt.ylabel("Number of parts")
# plt.title("Distribution of description lengths")
# plt.show()


# Parts with descriptions of length 0 to 4 (708 such parts)
# These are poor descriptions, and should be modified!

for_improvement = [x for x in parts if len(x["description"]) < 5]

# Parts with descriptions of length over 80 (758 such parts)
# These are good descriptions, and should be used for training!

for_training = [x for x in parts if len(x["description"]) > 80]

In [6]:
# Add website summaries to both these datasets

for i, p in enumerate(for_improvement):
    p["info"] = info(p["id"])
    print(i, p["id"])

0 BBa_C0090
1 BBa_C0400
2 BBa_C0420
3 BBa_C0600
4 BBa_C0620
5 BBa_C2030
6 BBa_C2031
7 BBa_C2032
8 BBa_C52934
9 BBa_I10010
10 BBa_I10101
11 BBa_I10652
12 BBa_I10777
13 BBa_I11000
14 BBa_I11010
15 BBa_I12205
16 BBa_I13700
17 BBa_I13701
18 BBa_I13702
19 BBa_I15006
20 BBa_I20533
21 BBa_I712002
22 BBa_I712024
23 BBa_I714005


Your max_length is set to 25, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


24 BBa_I7154
25 BBa_I716000
26 BBa_I716001
27 BBa_I716024
28 BBa_I716151
29 BBa_I716153
30 BBa_I716154
31 BBa_I716155
32 BBa_I716209
33 BBa_I716252
34 BBa_I716255
35 BBa_I716257
36 BBa_I716270
37 BBa_I716301
38 BBa_I716331
39 BBa_I716333
40 BBa_I716334
41 BBa_I720000
42 BBa_I722003
43 BBa_I723020
44 BBa_I723024
45 BBa_I723025
46 BBa_I725001
47 BBa_I725010
48 BBa_I726001
49 BBa_I726002
50 BBa_I726003
51 BBa_I726004
52 BBa_I729000
53 BBa_I732100


Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors


54 BBa_I732101
55 BBa_I732102
56 BBa_I732103
57 BBa_I732104
58 BBa_I732997
59 BBa_I741009
60 BBa_I741049
61 BBa_I741108
62 BBa_I742107
63 BBa_I745017
64 BBa_I745018
65 BBa_I766204
66 BBa_I766205
67 BBa_I766206
68 BBa_I766210
69 BBa_I769902
70 BBa_I769903
71 BBa_J01086
72 BBa_J01087
73 BBa_J01088
74 BBa_J01089
75 BBa_J01090
76 BBa_J01091
77 BBa_J05829
78 BBa_J07006
79 BBa_J07021
80 BBa_J07065
81 BBa_J125000
82 BBa_J125001
83 BBa_J17004
84 BBa_J176000
85 BBa_J176003
86 BBa_J176004
87 BBa_J176005
88 BBa_J176006
89 BBa_J176007
90 BBa_J176008
91 BBa_J176013
92 BBa_J176016
93 BBa_J176018
94 BBa_J176021
95 BBa_J176023
96 BBa_J176027
97 BBa_J176028
98 BBa_J176032
99 BBa_J176033
100 BBa_J176034
101 BBa_J176035
102 BBa_J176036
103 BBa_J176037
104 BBa_J18958
105 BBa_J18960
106 BBa_J212099
107 BBa_J212100
108 BBa_J212103
109 BBa_J22051
110 BBa_J22052
111 BBa_J29000
112 BBa_J29001
113 BBa_J29002
114 BBa_J290045
115 BBa_J29026
116 BBa_J29027
117 BBa_J29029
118 BBa_J29030
119 BBa_J29032
120 BBa_J2903

Your max_length is set to 25, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


398 BBa_K289002
399 BBa_K289003
400 BBa_K289005
401 BBa_K289007
402 BBa_K294060
403 BBa_K294120
404 BBa_K294206
405 BBa_K294354
406 BBa_K294898
407 BBa_K294902
408 BBa_K302006
409 BBa_K302007
410 BBa_K302008
411 BBa_K302009
412 BBa_K302010
413 BBa_K302011
414 BBa_K302019
415 BBa_K302023
416 BBa_K302024
417 BBa_K302025
418 BBa_K302026
419 BBa_K302027
420 BBa_K302028
421 BBa_K302029
422 BBa_K302032
423 BBa_K302033
424 BBa_K305000
425 BBa_K307101
426 BBa_K307102
427 BBa_K307105
428 BBa_K307106
429 BBa_K309099
430 BBa_K310018
431 BBa_K314011
432 BBa_K314444
433 BBa_K314560
434 BBa_K317035
435 BBa_K317998
436 BBa_K317999
437 BBa_K318516
438 BBa_K319033
439 BBa_K320015
440 BBa_K320016
441 BBa_K320999
442 BBa_K323104
443 BBa_K323129
444 BBa_K323137
445 BBa_K323140
446 BBa_K323141
447 BBa_K323142
448 BBa_K323143
449 BBa_K323144
450 BBa_K336054
451 BBa_K341453
452 BBa_K341717
453 BBa_K341718
454 BBa_K341719
455 BBa_K341720
456 BBa_K361010
457 BBa_K364202
458 BBa_K364300
459 BBa_K371037
460 BBa_

Your max_length is set to 25, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


465 BBa_K377700
466 BBa_K377703
467 BBa_K377704
468 BBa_K387008
469 BBa_K392029
470 BBa_K398100
471 BBa_K398200
472 BBa_K398201
473 BBa_K398300
474 BBa_K404245
475 BBa_K404249
476 BBa_K404307
477 BBa_K404316
478 BBa_K419007
479 BBa_K419009
480 BBa_K426003
481 BBa_K426006
482 BBa_K426007
483 BBa_K500005
484 BBa_K500006
485 BBa_K500007
486 BBa_K500008
487 BBa_K509005
488 BBa_K510034
489 BBa_K510035
490 BBa_K512000
491 BBa_K518005
492 BBa_K519004
493 BBa_K519005
494 BBa_K519010
495 BBa_K524204
496 BBa_K530001
497 BBa_K530002
498 BBa_K530025
499 BBa_K530026
500 BBa_K530027
501 BBa_K539413
502 BBa_K539513
503 BBa_K545005
504 BBa_K545006
505 BBa_K549010
506 BBa_K554003
507 BBa_K554004
508 BBa_K554005
509 BBa_K554009
510 BBa_K564000
511 BBa_K566027
512 BBa_K566028
513 BBa_K567020
514 BBa_K569011
515 BBa_K569017
516 BBa_K575003
517 BBa_K584017
518 BBa_K587000
519 BBa_K588000
520 BBa_K588001
521 BBa_K590080
522 BBa_K590081
523 BBa_K592002
524 BBa_K607015
525 BBa_K628004
526 BBa_K631039
527 BBa_

In [7]:
for i, p in enumerate(for_training):
    p["info"] = info(p["id"])
    print(i, p["id"])

0 BBa_C0060
1 BBa_C0080
2 BBa_E0033
3 BBa_E0035
4 BBa_E0040
5 BBa_E0044
6 BBa_E1010
7 BBa_G00200
8 BBa_I12040
9 BBa_I12046
10 BBa_I12047
11 BBa_I13035
12 BBa_I13940
13 BBa_I13941
14 BBa_I13942
15 BBa_I13943
16 BBa_I20248
17 BBa_I20249
18 BBa_I20250
19 BBa_I20251
20 BBa_I20257
21 BBa_I3000
22 BBa_I3001
23 BBa_I3002
24 BBa_I3420
25 BBa_I3421
26 BBa_I3422
27 BBa_I3423
28 BBa_I3424
29 BBa_I3425
30 BBa_I3521
31 BBa_I3522
32 BBa_I3600
33 BBa_I711022
34 BBa_I711034
35 BBa_I712016
36 BBa_I712028
37 BBa_I716183
38 BBa_I716184
39 BBa_I716377
40 BBa_I716378
41 BBa_I716380
42 BBa_I716381
43 BBa_I731014
44 BBa_I735010
45 BBa_I739007
46 BBa_I741016
47 BBa_I741017
48 BBa_I741107
49 BBa_I746008
50 BBa_I746101
51 BBa_I746103
52 BBa_I765002
53 BBa_I765003
54 BBa_J01025
55 BBa_J01056
56 BBa_J01062
57 BBa_J01102
58 BBa_J01105
59 BBa_J04700
60 BBa_J04800
61 BBa_J04900
62 BBa_J06551
63 BBa_J06571
64 BBa_J06574
65 BBa_J06610
66 BBa_J06611
67 BBa_J06612
68 BBa_J06628
69 BBa_J06629
70 BBa_J120014
71 BBa_J12002

Your max_length is set to 25, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


191 BBa_K1088002
192 BBa_K112203
193 BBa_K112204
194 BBa_K112205
195 BBa_K112211
196 BBa_K112212
197 BBa_K112217
198 BBa_K112223
199 BBa_K112234
200 BBa_K112246
201 BBa_K112248
202 BBa_K112361
203 BBa_K112362
204 BBa_K112363
205 BBa_K112369
206 BBa_K112370
207 BBa_K112371
208 BBa_K113021
209 BBa_K125000
210 BBa_K127003
211 BBa_K132025
212 BBa_K137083
213 BBa_K137084
214 BBa_K137085
215 BBa_K137086
216 BBa_K137087
217 BBa_K137088
218 BBa_K137089
219 BBa_K137090
220 BBa_K137091
221 BBa_K137101
222 BBa_K137102
223 BBa_K137107
224 BBa_K137108
225 BBa_K142024
226 BBa_K142025
227 BBa_K142026
228 BBa_K142027
229 BBa_K142028
230 BBa_K142029
231 BBa_K142030
232 BBa_K142031
233 BBa_K142048
234 BBa_K157001
235 BBa_K157010
236 BBa_K165091
237 BBa_K165101
238 BBa_K175006
239 BBa_K175034
240 BBa_K175035
241 BBa_K175050
242 BBa_K176026
243 BBa_K176028
244 BBa_K176029
245 BBa_K176030
246 BBa_K176031
247 BBa_K176032
248 BBa_K176036
249 BBa_K176080
250 BBa_K176089
251 BBa_K176090
252 BBa_K176091
253 BBa

In [8]:
# store the data with summaries
import json

x = json.dumps(for_improvement)
y = json.dumps(for_training)

# X = '[{"id": "BBa_I726001", "type": "Intermediate", "description": "I", "info": "iGEM 2007 Team Parts I luxI Usage and Biology Sequence and Features Assembly Compatibility: 10COMPATIBLE WITH RFC[10] . 12compatible with rfc[12"}, {"id": "BBa_I726002", "type": "Intermediate", "description": "R", "info": "iGEM 2007 Team Parts R luxR Usage and Biology Sequence and Features Assembly Compatibility: 10Compatible WITH RFC[10] 12COMPATIBLE WITH rfc[12] 21"}, {"id": "BBa_I741049", "type": "Regulatory", "description": " ", "info": "Sequence and Features Assembly Compatibility: 10Compatible with rfc[10] 12COMPATIBLE WITH RFC[12] 21Compatible with Rf[21] 23Compatiable with RFC"}, {"id": "BBa_I741108", "type": "Temporary", "description": "a", "info": "a Sequence and Features Assembly Compatibility: 10Compatible WITH RFC[10] 12COMPATIBLE WITH rfc[12] 21Compatible with rc[21] 23Compat"}, {"id": "BBa_J17004", "type": "Temporary", "description": "a", "info": "a a Sequence and Features Assembly Compatibility: 10COMPATIBLE WITH RFC[10] 12Compatible with rfc[12] 21INCOMPATIBIBLE WITH rc[21"}, {"id": "BBa_J29043", "type": "Reporter", "description": "a", "info": "a a Sequence and Features Assembly Compatibility: 10INCOMPATIBLE WITH RFC[10]Illegal EcoRI site found at 316 12INCOMPATIBILITY WITH rfc["}, {"id": "BBa_K245138", "type": "Coding", "description": "x", "info": "x x Sequence and Features Assembly Compatibility: 10compatible with rfc[10] 12compatible with RFC[12] 21compatibile with Rf[21] 23compat"}, {"id": "BBa_K252015", "type": "RBS", "description": "f", "info": "f ff sequence and features assembly compatibility: 10compatible with rfc[10] 12compatible with RFC[12] 21compatible with regc[21] 23compat"}, {"id": "BBa_K361010", "type": "Regulatory", "description": " ", "info": "Sequence and Features Assembly Compatibility: 10Compatible WITH RFC[10] 12COMPATIBLE WITH rfc[12] 21Compatible with rc[21] 23Compatiable with "}, {"id": "BBa_K404245", "type": "Project", "description": " ", "info": "Sequence and Features Assembly Compatibility: 10Compatible WITH RFC[10] 12COMPATIBLE WITH rfc[12] 21Compatible with rc[21] 23Compatiable with "}, {"id": "BBa_K404249", "type": "Project", "description": " ", "info": "Sequence and Features Assembly Compatibility: 10Compatible WITH RFC[10] 12COMPATIBLE WITH rfc[12] 21Compatible with rc[21] 23Compatiable with "}, {"id": "BBa_K404307", "type": "Composite", "description": " ", "info": "x Sequence and Features Assembly Compatibility: 10Compatible WITH RFC[10] 12COMPATIBLE WITH rfc[12] 21Compatible with rc[21] 23Compatiable"}, {"id": "BBa_K519004", "type": "Composite", "description": "a", "info": "a a Sequence and Features Assembly Compatibility: 10Compatible with rfc[10] 12INCOMPATIBLE WITH RFC[12]Illegal nheI site found at 7I"}, {"id": "BBa_K519005", "type": "Composite", "description": "a", "info": "a a Sequence and Features Assembly Compatibility: 10Compatible with rfc[10] 12INCOMPATIBLE WITH RFC[12]Illegal nheI site found at 7I"}, {"id": "BBa_K879927", "type": "Plasmid", "description": " ", "info": "this is a plasmid backbone that was designed to have its copy number controlled by the presence or absence of the inducer molecule IPTG . this copy number corresponds to the expression level of the genes on the "}, {"id": "BBa_K879928", "type": "Plasmid", "description": " ", "info": "this biobrick has been tested in an experiment looking at plasmid loss from a culture of cells that are not growing under selection . the culture containing IPTG produced approximately 25 times as many colonies as the culture without IP"}]'
# Y = '[{"id": "BBa_J01062", "type": "Intermediate", "description": "Mobilized OriTF with Lock1ed cI --%3E pRM --%3E GFP %3D %5BOriTF%5D%5BOnLock1%5D%5BcI%5D%5BDblTerminator%5D%5BpRM GFP%5D", "info": "Mobilized OriTF with Lock1ed cI--> pRM --> GFP = [OriTF][OnLock1][cI][dblTerminator]["}, {"id": "BBa_J23040", "type": "Signalling", "description": "%5BTetR%5D%5Brbs%5D%5BLuxR%5D%5BdblTerm%5D%5BLuxPR%5D+%5Brbs%5D%5BLacI%5D%5BdblTerm%5D  %22AHL-dependent inverter%22", "info": "F2620+P0412 sequence and features assembly compatibility: 10Compatible with rfc[10] 12Compatible with RFC[12] 21INCOMPATIBLE WITH RFC [21]"}, {"id": "BBa_J72103", "type": "Composite", "description": "%7BaraC-Pbad%7D%7Brbs.coi%7D%7Bdouble terminator%7D%7Bcin, no start, frameshift%7D%7BrepL no start%7D%7BLPsit%7D%7Brbs.pacA%7D%7BT", "info": "this part is in BBb Format . it is flanked by BglII and BamHI sites instead of XbaI and SpeI . the phagemid is referred to internally as"}, {"id": "BBa_J72116", "type": "Composite", "description": "%7BaraC-Pbad%7D%7Brbs.coi%7D%7Bdouble terminator%7D%7Bcin, no start, frameshift%7D%7BLPsit%7D%7Brbs.pacA%7D%7BTrnpB%7D", "info": "this part is in BBb Format . it is flanked by BglII and BamHI sites instead of XbaI and SpeI . the part is referred to internally as jt"}, {"id": "BBa_K078004", "type": "Coding", "description": "2%26%2365292%3B2%91%26%2365292%3B3-Trihydroxybiphenyl 1,2-dioxygenase. The second step enzyme in dixon degradation", "info": "there is currently no text in this page . you can search for this page title in other pages, or search the related logs, but you do not have permission to create this page. if you want to create a new page"}, {"id": "BBa_K078005", "type": "Coding", "description": "2%26%2365292%3B2%91%26%2365292%3B3-Trihydroxybiphenyl 1,2-dioxygenase. The second step enzyme in dixon degradation", "info": "2,2\\u2018,3-trihydroxybiphenyl 1,2-dioxygenase is one enzyme in dioxin degradation pathway . it is one kind of estradoil dioxygenese repon"}, {"id": "BBa_K531007", "type": "Composite", "description": "P%3Csub%3ErsaA%3C/sub%3E constitutive promoter, %3Ci%3Eesp%3C/i%3E optimized for %3Ci%3ECaulobacter%3C/i%3E, and %3Ci%3ErsaA%3C/i%3E", "info": "most of our chimeric RsaA-fusion proteins were expressed and secreted from Caulobacter . both DspB and Esp constructs with pxyl inducible promoter and chi"}, {"id": "BBa_K531008", "type": "Composite", "description": "P%3Csub%3Exyl%3C/sub%3E + %3Ci%3ECaulobacter%3C/i%3E optimized %3Ci%3EdspB%3C/i%3E + %3Ci%3ErsaA%3C/i%3E C-term", "info": "most of our chimeric RsaA-fusion proteins were expressed and secreted from Caulobacter . both DspB and Esp constructs with Pxyl inducible promoter can inhibit the bio"}, {"id": "BBa_K531009", "type": "Composite", "description": "P%3Csub%3ErsaA%3C/sub%3E + %3Ci%3ECaulobacter%3C/i%3E optimized %3Ci%3EdspB%3C/i%3E + %3Ci%3ErsaA%3C/i%3E C-term", "info": "most of our chimeric RsaA-fusion proteins were expressed and secreted from Caulobacter . both DspB and Esp constructs with Pxyl inducible promoter can inhibit the bio"}, {"id": "BBa_K531010", "type": "Composite", "description": "Constitutive promoter and RBS BBa_K081005 + %3Ci%3ECaulobacter%3C/i%3E optimized %3Ci%3EdspB%3C/i%3E + %3Ci%3ErsaA%3C/i%3E C", "info": "most of our chimeric RsaA-fusion proteins were expressed and secreted from Caulobacter . both DspB and Esp constructs with Pxyl inducible promoter can inhibit the bio"}, {"id": "BBa_K531011", "type": "Composite", "description": "Constitutive promoter and RBS BBa_K081005 + %3Ci%3ECaulobacter%3C/i%3E optimized %3Ci%3Eesp%3C/I%3E + C-terminal sec", "info": "most of our chimeric RsaA-fusion proteins were expressed and secreted from Caulobacter . both DspB and Esp can inhibit the biofilm growth . biofilm assay data showed a"}, {"id": "BBa_K733007", "type": "Composite", "description": "%3Ci%3EPveg%3C/i%3E + spoVG RBS + %3Ci%3ElytC%3C/i%3E + linker + RPMrel + consensus RBS + GFP + double terminator", "info": "project seeks to design recombinant bacteria that specifically target and suppress the growth of colorectal carcinoma cells in a controllable way . our proposed solution requires the phage display peptide \\u2018RPM"}, {"id": "BBa_K733012", "type": "Composite", "description": "%3Ci%3ExylR%3C/i%3E+%3Ci%3EPxylA%3C/i%3E+RBS+%3Ci%3EydcE%3C/i%3E+%3Ci%3EPtms%3C/i%3E+RBS+%3Ci%3EydcD%3C/i%3E%3A Cell Growth Inhibition Device", "info": "ptms+RBS+ydcD is for the stabilization of the cell growth inhibition system . the rationale for including this growth inhibition device is that over-dose of BMP2 can cause unexpected proliferation of"}, {"id": "BBa_K823023", "type": "Plasmid_Backbone", "description": "pSB%3Csub%3EBs%3C/sub%3E1C%3A Empty backbone for integration into %3Ci%3EBacillus subtilis%3C/i%3E %3Ci%3EamyE%3C/i%3E locus", "info": "pSBBs1C is an empty backbone vector for the usage in Bacillus subtilis . it integrates in the amyE locus and can be selected with chloramphenicol (cat gene) this"}, {"id": "BBa_K823024", "type": "Plasmid_Backbone", "description": "pSB%3Csub%3EBs%3C/sub%3E4S-P%3Ci%3E%3Csub%3Exyl%3C/i%3E%3C/sub%3E%3A Integrative expression vector for %3Ci%3EBacillus subtilis%3C/i", "info": "pSBBs4S-Pxyl is an expression vector for Bacillus subtilis . it integrates at the thrC locus and has a chloramphenicol resistance for selection in B"}, {"id": "BBa_K823026", "type": "Plasmid_Backbone", "description": "pSB%3Csub%3EBs%3C/sub%3E0K-P%3Csub%3Espac%3C/sub%3E (replicative Bacillus subtilis expression vector%3B IPTG inducible", "info": "pSBBs0K-Pspac is a replicative expression vector for Bacillus subtilis . it has an ampicillin resistance for cloning in E.coli and kanamycin resistance for selection"}, {"id": "BBa_K823030", "type": "Regulatory", "description": "P%3Csub%3E%3Ci%3EcotYZ%3C/i%3E%3C/sub%3E%3A %3Ci%3EB. subtilis%3C/i%3E promoter regulating expression of spore crust proteins", "info": "PcotYZ lies within cotVWXYZ gene cluster, which regulates expression of CotYZ . to test the activity we cloned this promoter into the lux-reporter vector"}, {"id": "BBa_K823033", "type": "Regulatory", "description": "P%3Csub%3E%3Ci%3EcotV%3C/i%3E%3C/sub%3E%3A  %3Ci%3EB. subtilis%3C/i%3E promoter regulates spore crust protein cotV", "info": "PcotV lies within cotVWXYZ gene cluster, which regulates expression of spore crust protein CotV . to test the activity we cloned this promoter into the lux-re"}, {"id": "BBa_M10220", "type": "Composite", "description": "%7BPbad.rbs.prepro.StrepTag%7D%7B%26%23706%3BGS5-IILK%26%23707%3B%7D%7B%26%23706%3BupaG_short%21%7D%7BdblTerm%7D", "info": "recombinant leucine zipper constructs were tested for their ability to bind streptavidin . the constructs did not appear to be able to clump at the bottom of the LB plates ."}, {"id": "BBa_M10221", "type": "Device", "description": "%7BPbad.rbs.prepro.StrepTag%7D%7D%7B%26%23706%3BGS5-IILK%26%23707%3B%7D%7B%26%23706%3BAg43_short%21%7D%7BdblTerm%7D", "info": "toxicity can be inferred from the differences in growth rate (as determined by changes in OD 600 over time) between samples containing LB and LB+arabinose . pbca9145-b"}, {"id": "BBa_M10222", "type": "Device", "description": " %7BPbad.rbs.prepro.StrepTag%7D%7B%26%23706%3BGS5-IILK%26%23707%3B%7D%7B%26%23706%3BespP(beta)%21%7D%7BdblTerm%7D ", "info": "recombinant leucine zipper constructs were tested for their ability to bind streptavidin . the constructs did not appear to clump in the presence of arabinose . a method for quantify"}, {"id": "BBa_S03679", "type": "Intermediate", "description": "(2,-1) RFP%3Csub%3Erev%3C/sub%3E-RBS%3Csub%3Erev%3C/sub%3E-HixC-RBS-TetF-HixC %3A pLac%3Csub%3Erev%3C/sub%3E-HixC", "info": "RFPrev-RBSrev-HixC : pLacrev-hixc Construction intermediate Sequence and Features Assembly Compatibility: 10COMPATIBLE WITH RFC[10] 12INCOM"}, {"id": "BBa_S03680", "type": "Intermediate", "description": "(-2,1) RFP%3Csub%3Erev%3C/sub%3E-RBS%3Csub%3Erev%3C/sub%3E-HixC-TB-RBS%3Csub%3Erev%3C/sub%3E-HixC %3A pLac-HixC", "info": "RFPrev-RBSrev-HixC : pLac-hixc Construction intermediate Sequence and Features Assembly Compatibility: 10COMPATIBLE WITH RFC[10] 12INCOMPA"}, {"id": "BBa_S03681", "type": "Intermediate", "description": "(-2,-1) RFP%3Csub%3Erev%3C/sub%3E-RBS%3Csub%3Erev%3C/sub%3E-HixC-TB-RBS%3Csub%3Erev%3C/sub%3E-HixC %3A pLac%3Csub%3Erev%3C/sub%3E-HixC", "info": "RFPrev-RBSrev-HixC : pLacrev-hixc Construction intermediate Sequence and Features Assembly Compatibility: 10COMPATIBLE WITH RFC[10] 12INCOM"}, {"id": "BBa_S03685", "type": "Intermediate", "description": "(1,-2) RFP%3Csub%3Erev%3C/sub%3E-RBS%3Csub%3Erev%3C/sub%3E-HixC-pLac-HixC %3A TetB-RBS%3Csub%3Erev%3C/sub%3E-HixC", "info": "RFPrev-RBSrev-HixC-pLac-hixc : construction intermediate Sequence and features Assembly Compatibility: 10COMPATIBLE WITH RFC[10] 12INCOMPA"}, {"id": "BBa_S03687", "type": "Intermediate", "description": "(-1,2) RFP%3Csub%3Erev%3C/sub%3ERBS%3Csub%3Erev%3C/sub%3E-HixC-pLac%3Csub%3Erev%3C/sub%3E-HixC %3A RBS-TetF-HixC", "info": "RFPrevRBSrev-HixC-pLacrev-hixc : RBS-TetF-hxc Construction intermediate Sequence and features Assembly Compatibility: 10COMPAT"}, {"id": "BBa_S03688", "type": "Intermediate", "description": "(-1,-2) RFP%3Csub%3Erev%3C/sub%3ERBS%3Csub%3Erev%3C/sub%3E-HixC-pLac%3Csub%3Erev%3C/sub%3E-HixC %3A TB-RBS%3Csub%3Erev%3C/sub%3E-HixC", "info": "TB-RBSrev-HixC-pLacrev-hixc Construction intermediate Sequence and features Assembly Compatibility: 10Compatible WITH RFC[10] 12INCOMPATIBLE WITH R"}]'

# for_improvement = json.loads(X)
# for_training = json.loads(Y)

print(x, y, sep="\n\n")

[{"id": "BBa_C0090", "type": "Coding", "description": "smaI", "info": "smaI Coding Region for AHL synthase smaI from Serratia"}, {"id": "BBa_C0400", "type": "Coding", "description": "BglG", "info": "BglG is a protein that is known to bind to terminator loops in bgl"}, {"id": "BBa_C0420", "type": "Coding", "description": "BglF", "info": "BglF Sensor permease of Bgl operon Sequence and Features Assembly Compatibility: 10"}, {"id": "BBa_C0600", "type": "Coding", "description": "Tar", "info": "Tar Chemotaxis aspartate receptor Usage and Biology Sequence and Features Assembly Compatibility: 10COM"}, {"id": "BBa_C0620", "type": "Coding", "description": "Tsr", "info": "Tsr Chemotaxis serine receptor sequence and features assembly compatibility: 10COMPATIBLE WITH R"}, {"id": "BBa_C2030", "type": "Coding", "description": "Fos", "info": "Fos Encodes the Fos leucine zipper domain Sequence and Features Assembly Compatibility: 10COM"}, {"id": "BBa_C2031", "type": "Coding", "description": "JunB", "i

In [9]:
training_examples = []
shots = []

for p in for_training:
    inp = f"Type: {p['type']}, Information: {p['info']}"
    outp = p["description"]
    
    training_examples.append({"input": inp, "target": outp})
    shots.append(f"{inp}, Description: {outp}")

In [10]:
class DescriptionDataset(Dataset):
    def __init__(self, tokenizer, examples):
        self.examples = examples
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        inputs = self.tokenizer.encode(example["input"] + ", Description: ", return_tensors="pt", padding=True, truncation=True, max_length=512)
        targets = self.tokenizer.encode(example["target"], return_tensors="pt", padding=True, truncation=True, max_length=512)
        return {"input_ids": inputs, "attention_mask": inputs.ne(0), "labels": targets}

    def collate_fn(self, batch):
        inputs = pad_sequence([item["input_ids"].squeeze(0) for item in batch], batch_first=True)
        attention_masks = pad_sequence([item["attention_mask"].squeeze(0) for item in batch], batch_first=True)
        labels = pad_sequence([item["labels"].squeeze(0) for item in batch], batch_first=True, padding_value=self.tokenizer.pad_token_id)
        
        return {"input_ids": inputs, "attention_mask": attention_masks, "labels": labels}

In [11]:
# Prepare the dataset and data loader
dataset = DescriptionDataset(tokenizer, training_examples)
batch_size = 4  # Adjust this based on your resources
train_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=True)

# Fine-tuning settings
num_epochs = 3
learning_rate = 5e-5

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Fine-tuning loop
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids = batch["input_ids"].squeeze(1).to(model.device)
        attention_mask = batch["attention_mask"].squeeze(1).to(model.device)
        labels = batch["labels"].squeeze(1).to(model.device)
            
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_description_model")
tokenizer.save_pretrained("fine_tuned_description_model")

('fine_tuned_description_model/tokenizer_config.json',
 'fine_tuned_description_model/special_tokens_map.json',
 'fine_tuned_description_model/spiece.model',
 'fine_tuned_description_model/added_tokens.json',
 'fine_tuned_description_model/tokenizer.json')

In [12]:
# Generate descriptions
for p in for_improvement:
#     one_shot = shots[0] + "\n"
    
#     input_text = one_shot + f"Type: {p['type']}, Information: {p['info']}"
    input_text = f"Type: {p['type']}, Information: {p['info']}, Description: "
    
    input_ids = tokenizer.encode(input_text + ", Description: ", return_tensors="pt")
    output = model.generate(input_ids, max_length=60, num_beams=4, early_stopping=True)
    description = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"ID: {p['id']}")
    print(f"Original Description: {p['description']}")
    print(f"Description: {description}")
    print("--------------")

ID: BBa_C0090
Original Description: smaI
Description: smaI coding region for AHL synthase smaI from Serratia
--------------
ID: BBa_C0400
Original Description: BglG
Description: BglG is a protein that is known to bind to terminator loops in bgl
--------------
ID: BBa_C0420
Original Description: BglF
Description: BglF Sensor permease of Bgl operon Sequence and Features Assembly Compatibility:
--------------
ID: BBa_C0600
Original Description: Tar
Description: Chemotaxis aspartate receptor (%3Ci%3Easpart) %3C/i%3E Usage and Biology Sequence and Features Assembly Compatibility: 10COM%3C/i%3E
--------------
ID: BBa_C0620
Original Description: Tsr
Description: Tsr Chemotaxis serine receptor sequence and features assembly compatibility%3A compatibility%3A with RBS
--------------
ID: BBa_C2030
Original Description: Fos
Description: %7BPos%7D%7Bleucine%7D%7D%7Bzip%7D%7B%7D%7B%7D%7B%7D%7B%7D%7B%7D%
--------------
ID: BBa_C2031
Original Description: JunB
Description: JunB Encodes the JunB leucin